In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy as np
import sys
from keras.preprocessing.text import Tokenizer
import string
import re

Using TensorFlow backend.


In [2]:
file_names=['AIIMAT.txt', 'MLOE.txt','OKEWFSMP.txt','TAM.txt','TAMatter.txt', 'THWP.txt','TPP.txt']
corpus=[]
def get_corpus(path):
    with open(path,'r', errors= 'ignore') as data:
        corpus=data.read()
        corpus=corpus.lower()
    return corpus

for i in range(len(file_names)):
    path='/Users/phuongqn/Desktop/INF552/Homework/Homework 7 Data/Book Files/books/'+file_names[i]+''
    corpus.append(get_corpus(path))

In [3]:
def clean_text(text):
    text = text.translate(string.punctuation)
    
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=:;<]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"<", " ", text)
    text = re.sub(r";", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"[0123456789]", " ", text)
    
    return text

In [ ]:
c0 = clean_text(corpus[0])
c1 = clean_text(corpus[1])
c2 = clean_text(corpus[2])
c3 = clean_text(corpus[3])
c4 = clean_text(corpus[4])
c5 = clean_text(corpus[5])
c6 = clean_text(corpus[6])
corpus_c=[c0,c1,c2,c3,c4,c5,c6]
t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=False, split=' ', char_level=True, oov_token=None, document_count=0)
t.fit_on_texts(corpus_c)
t.word_index

In [5]:
chars=[]
for i in corpus_c:
    chars.extend(list(set(set(i))))
    
chars = sorted(list(set(chars)))
char_int = dict((c, i) for i, c in enumerate(chars))
int_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
char_int

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [7]:
def i_o(file, size):
    ip=[]
    op=[]
    for i in range(0, len(file) - size+1, 1):
        seq_in = file[i:i + size - 1]
        seq_out = file[i + size-1]
        ip.append([char_int[c] for c in seq_in])
        op.append(char_int[seq_out])
    return ip, op

In [8]:
ip_seq=[]
op_char=[]
for c in corpus_c:
    w=100
    ip, op = i_o(c, w)
    ip_seq.extend(ip)
    op_char.extend(op)

In [9]:
# reshape X
X = np.reshape(ip_seq, (len(ip_seq), 99,1))
# normalize
X = X / float(len(chars))
# one hot encode the output variable
y = np_utils.to_categorical(op_char)
print(X.shape)

(4850971, 99, 1)


In [10]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Here I noticed that using an LSTM with N=256 will typically yield better results, but the run time is long (4hrs per epoch). I have attached that version, which I stopped prematurely, for comparison!

In [14]:
LSTMmodel = Sequential()
LSTMmodel.add(LSTM(28, input_shape=(X.shape[1], X.shape[2])))
# LSTMmodel.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), dropout=0.2))
LSTMmodel.add(Dropout(0.2))
LSTMmodel.add(Dense(y.shape[1], activation='softmax'))
print(LSTMmodel.summary())
LSTMmodel.compile(loss='categorical_crossentropy', optimizer='adam')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 28)                3360      
_________________________________________________________________
dropout_1 (Dropout)          (None, 28)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 27)                783       
Total params: 4,143
Trainable params: 4,143
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
filepath= '/Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [16]:
LSTMmodel.fit(X, y, epochs=30, batch_size=64, callbacks=callbacks_list)

Train on 4850971 samples
Epoch 1/30
4850880/4850971 [============================>.] - ETA: 0s - loss: 2.5233
Epoch 00001: loss improved from inf to 2.52329, saving model to /Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-01-2.5233.hdf5
4850971/4850971 [==============================] - 2431s 501us/sample - loss: 2.5233
Epoch 2/30
4850944/4850971 [============================>.] - ETA: 0s - loss: 2.3643
Epoch 00002: loss improved from 2.52329 to 2.36433, saving model to /Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-02-2.3643.hdf5
4850971/4850971 [==============================] - 2408s 496us/sample - loss: 2.3643
Epoch 3/30
4850944/4850971 [============================>.] - ETA: 0s - loss: 2.3006
Epoch 00003: loss improved from 2.36433 to 2.30056, saving model to /Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-03-2.3006.hdf5
4850971/4850971 [==============================] - 2391s 493us/sample - loss: 2.3006
E

4850880/4850971 [============================>.] - ETA: 0s - loss: 2.1477
Epoch 00026: loss improved from 2.14967 to 2.14774, saving model to /Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-26-2.1477.hdf5
4850971/4850971 [==============================] - 2361s 487us/sample - loss: 2.1477
Epoch 27/30
4850880/4850971 [============================>.] - ETA: 0s - loss: 2.1468
Epoch 00027: loss improved from 2.14774 to 2.14681, saving model to /Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-27-2.1468.hdf5
4850971/4850971 [==============================] - 2699s 556us/sample - loss: 2.1468
Epoch 28/30
4850944/4850971 [============================>.] - ETA: 0s - loss: 2.1449
Epoch 00028: loss improved from 2.14681 to 2.14491, saving model to /Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-28-2.1449.hdf5
4850971/4850971 [==============================] - 3178s 655us/sample - loss: 2.1449
Epoch 29/30
4850880/4850971 [==

In [17]:
weight='/Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-30-2.1433.hdf5'
LSTMmodel.load_weights(weight)
LSTMmodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [20]:
seed = 'There are those who take mental phenomena naively, just as they would physical phenomena This school of psychologists tends not to emphasize the object'

pattern = [char_int[c] for c in seed[-99:].lower()]

for i in range(1000):
    seq = np.reshape(pattern, (1, len(pattern), 1))
    seq = seq / float(len(chars))
    charpredict = LSTMmodel.predict(seq, verbose=0)
    ind = np.argmax(charpredict)
    seed+=int_char[ind]
    pattern.append(ind)
    pattern = pattern[1:len(pattern)]

print(seed)

There are those who take mental phenomena naively, just as they would physical phenomena This school of psychologists tends not to emphasize the object the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so the so t